# Evaluation TextGrad with TextualVerifier in Loss

In [1]:
import re
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.loss import TextLoss
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierWithTracker


## Load Dataset

In [2]:
initial_solution = pd.read_csv("datasets/initial-solution.csv")
initial_solution

,id,formatted_question,raw_solution,correct_answer,source,subject
0,2,Answer the following multiple choice question....,Here's how we can determine the number of carb...,A,GPQA-Diamond,-
1,4,Answer the following multiple choice question....,Maxwell's equations in our universe are:\n\n1....,A,GPQA-Diamond,-
2,8,Answer the following multiple choice question....,Here's how we can analyze the results and dete...,B,GPQA-Diamond,-
3,1,Answer the following multiple choice question....,The energy-time uncertainty principle states t...,A,GPQA-Diamond,-
4,22,Answer the following multiple choice question....,The question asks about the oxidizing power of...,D,GPQA-Diamond,-
...,...,...,...,...,...,...
407,394,Answer the following multiple choice question....,The police car is moving towards the wall. Le...,B,MMLU-CP,college_physics
408,384,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,A,MMLU-CP,college_physics
409,404,Answer the following multiple choice question....,The diffraction of electrons by a crystal latt...,A,MMLU-CP,college_physics
410,390,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,D,MMLU-CP,college_physics


## Setup Engine

In [3]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

/Users/eugeniusms/.pyenv/versions/3.11.6/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiment Iterations

In [4]:
def extract_answer(text):
    # Now extract correctly
    match = re.search(r"(?i)Answer\s*:\s*([A-D])", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return "Z" # Z -> means for None (Incorrect Answer)

In [5]:
def extract_answer_llm(question, final_solution):
    prompt = f"""
    <Question>
    {question}
    </Question>
    
    <Solution>
    {final_solution}
    </Solution>

    <Task>
    Based on last step of solution, which letter (ABCD) selected in question options?
    Response MUST ONLY in 1 letter where LETTER is one of ABCD!
    </Task>
    """
    answer = engine.generate(prompt)
        
    return answer

In [6]:
def evaluate(row_data, engine):
    try:
        match = initial_solution[initial_solution["id"] == row_data["id"]]
        if match.empty:
            return None  # or raise error
        formatted_question = match.iloc[0]["formatted_question"]
        
        solution = Variable(row_data["raw_solution"],
                            requires_grad=True,
                            role_description=f"Solution to the math question: {formatted_question}")
        loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
                                        Do not attempt to solve it yourself, do not give a solution, 
                                        only identify errors. Be super concise.""",
                                        requires_grad=False,
                                        role_description="system prompt")
        verification_task_prompts = [
            """
            1. When using $instruction to $instance, I got the loss $calculation.
            2. Evaluate the loss $calculation value correctly reflects the performance of the instance.
            3. If the loss $calculation is incorrect or inconsistent, provide the corrected version of the loss $calculation. 
            4. Do NOT calculate the solution/instance, evaluate $calculation ONLY.
            """
        ]

        optimizer = TextualGradientDescent([solution])
        loss = TextLoss(loss_system_prompt, engine=engine)
        loss_value = loss(solution) # Forward method in Loss Function
        original_loss = loss_value.value

        # TextualVerifier
        verifier = TextualVerifierWithTracker(
            verifier_engine=engine, 
            use_cot_generation=True,
            use_step_breakdown=True,
            verification_task_prompts=verification_task_prompts,
            enable_logging=False,
        )

        verified_result = verifier.verify(instance=solution,
                                        instruction=loss_system_prompt,
                                        calculation=loss_value)

        tracker_data = verifier.get_tracker() # Tracker data     

        loss_value.set_value(verified_result.value) 
        
        loss_value.backward()
        optimizer.step()

        final_solution_answer = extract_answer(solution.value)
        if final_solution_answer == "Z":
            final_solution_answer = extract_answer_llm(
                question=formatted_question, 
                final_solution=solution.value)

        return {
            "id": row_data["id"],
            "source": row_data["source"],
            "subject": row_data["subject"],
            "question": formatted_question,
            "correct_answer": row_data["correct_answer"],
            "original_solution": row_data["raw_solution"],
            "original_loss": original_loss,
            "verified_loss": verified_result,
            "optimized_solution": solution.value,
            "verified_optimized_solution": "",
            "final_solution": solution.value,
            "final_solution_answer": final_solution_answer,
            "success": True,
            "error_message": None,
            "processing_time_ms": tracker_data['processing_time_ms'],
            "total_llm_calls": tracker_data['total_llm_calls'],
            "total_input_tokens": tracker_data['total_input_tokens'],
            "total_output_tokens": tracker_data['total_output_tokens'],
            "result": tracker_data
        }
        
    except Exception as e:
        # Return error information
        return {
            "id": row_data["id"],
            "source": "",
            "subject": "",
            "question": "",
            "correct_answer": "",
            "original_solution": "",
            "original_loss": "",
            "verified_loss": "",
            "optimized_solution": "",
            "verified_optimized_solution": "",
            "final_solution": "",
            "final_solution_answer": "", 
            "success": False,
            "error_message": str(e),
            "processing_time_ms": 0,
            "total_llm_calls": 0,
            "total_input_tokens": 0,
            "total_output_tokens": 0,
            "result": {}
        }

In [7]:
def run_evaluation():
    results = []
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = [
            executor.submit(evaluate, row.to_dict(), engine) 
            for _, row in initial_solution.iterrows()  
        ]
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
            try:
                result = future.result(timeout=None)
                if result is not None:
                    results.append(result)
            except Exception as e:
                print(f"Task failed: {e}")
                error_result = {
                    'success': False,
                    'error_message': str(e),
                    'result': {}
                }
                results.append(error_result)

    experiment_df = pd.DataFrame(results)
    print(f"Completed in {time.time() - start_time:.1f} seconds")
    experiment_df.to_csv('results/textgrad-tv-l-1v-step.csv', index=False)
    
    return experiment_df

In [8]:
run_evaluation()

I0000 00:00:1750146047.653539 35633615 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
Processing:   0%|          | 0/412 [00:00<?, ?it/s]

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> The ribonucleoprotein particle is part of the ribosome.  The nascent chain is the newly forming protein. The ribosome is where proteins are synthesized. "You really need some sugar" likely refers to glycosylation, a modification that often happens in the endoplasmic reticulum (ER) and Golgi.  "Goodbye; I need to be on my way" suggests the protein is leaving the ribosome for its final destination.  Proteins synthesized on ribosomes attached to the ER enter the ER lumen for further processing and folding.  From the ER, proteins can go to various locations, including the Golgi, lysosomes, or be secreted.  The proteasome is where proteins are degraded, not their des

Processing:   1%|          | 3/412 [00:11<21:15,  3.12s/it]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> The potential is given by:\nV(r, θ) = (1/2)kr^2 + (3/2)kr^2 cos^2(θ)\nV(r, θ) = (1/2)kr^2 (1 + 3cos^2(θ))\n\nWe can rewrite this in Cartesian coordinates:\nx = rcos(θ)\ny = rsin(θ)\nr^2 = x^2 + y^2\ncos^2(θ) = x^2 / (x^2 + y^2)\n\nV(x, y) = (1/2)k(x^2 + y^2) + (3/2)k(x^2 + y^2) * (x^2 / (x^2 + y^2))\nV(x, y) = (1/2)k(x^2 + y^2) + (3/2)kx^2\nV(x, y) = (1/2)ky^2 + (1/2)kx^2 + (3/2)kx^2\nV(x, y) = (1/2)ky^2 + 2kx^2\nV(x, y) = (1/2)ky^2 + (1/2)(4k)x^2\n\nThis potential represents two independent harmonic oscillators in x and y directions.\nThe potential for the x-direction is V(x) = (1/2)(4k)x^2, and for the y-direction is V(y) = (1/2)ky^2.\n\nThe energy levels for 

Processing:   1%|          | 4/412 [00:12<15:51,  2.33s/it]

...

Processing:  99%|█████████▉| 407/412 [04:43<00:07,  1.54s/it]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> Here's how to figure out which harmonics survive when an open pipe is closed at one end:\n\n1. **Open pipe harmonics:** An open pipe (open at both ends) produces all integer multiples of the fundamental frequency.  In this case, the fundamental frequency is 50 Hz, and we see harmonics at 2x50 Hz, 3x50 Hz, 4x50 Hz, and 5x50 Hz.\n\n2. **Closed pipe harmonics:** A closed pipe (closed at one end) only produces odd integer multiples of the fundamental frequency.  Let's call the new fundamental frequency f'.\n\n3. **Relating the frequencies:** When you close one end of the pipe, the fundamental frequency becomes half the original fundamental frequency.  So, f' = 50 Hz

Processing: 100%|██████████| 412/412 [05:02<00:00,  1.36it/s]


Completed in 302.6 seconds


,id,source,subject,question,correct_answer,original_solution,original_loss,verified_loss,optimized_solution,verified_optimized_solution,final_solution,final_solution_answer,success,error_message,processing_time_ms,total_llm_calls,total_input_tokens,total_output_tokens,result
0,27,GPQA-Diamond,-,Answer the following multiple choice question....,A,The ribonucleoprotein particle is part of the ...,No errors identified.\n,<VERIFIED>Not applicable.\n</VERIFIED>,The ribonucleoprotein particle is part of the ...,,The ribonucleoprotein particle is part of the ...,D,True,None,2306.972900,2,375,12,{'setup': {'verifier_engine': '<textgrad.engin...
1,36,GPQA-Diamond,-,Answer the following multiple choice question....,C,Here's how we can determine the most likely ob...,No errors.\n,<VERIFIED>No errors.\n</VERIFIED>,Here's how we can determine the most likely ob...,,Here's how we can determine the most likely ob...,C,True,None,3389.712646,2,487,9,{'setup': {'verifier_engine': '<textgrad.engin...
2,11,GPQA-Diamond,-,Answer the following multiple choice question....,D,Let's analyze each statement:\n\nA. This state...,No errors identified.\n,<VERIFIED>The provided answer correctly identi...,Let's analyze each statement:\n\nA. This state...,,Let's analyze each statement:\n\nA. This state...,D,True,None,4094.871826,2,529,46,{'setup': {'verifier_engine': '<textgrad.engin...
3,7,GPQA-Diamond,-,Answer the following multiple choice question....,A,The process described is pair production from ...,No errors.\n,<VERIFIED>No errors.\n</VERIFIED>,The process described is pair production from ...,,The process described is pair production from ...,A,True,None,3527.519043,2,509,9,{'setup': {'verifier_engine': '<textgrad.engin...
4,32,GPQA-Diamond,-,Answer the following multiple choice question....,D,Here's how we can determine the energy of the ...,1. **Incorrect rest mass:** The rest mass of ⁶...,<VERIFIED>The rest mass of ⁶Li is calculated b...,Here's how we can determine the energy of the ...,,Here's how we can determine the energy of the ...,D,True,None,10889.475098,5,2394,178,{'setup': {'verifier_engine': '<textgrad.engin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,404,MMLU-CP,college_physics,Answer the following multiple choice question....,A,The diffraction of electrons by a crystal latt...,The final answer is incorrectly rounded. 23.5 ...,<VERIFIED>23.5 eV\n</VERIFIED>\n<VERIFIED>23.5...,The diffraction of electrons by a crystal latt...,,The diffraction of electrons by a crystal latt...,A,True,None,9707.145996,7,4004,140,{'setup': {'verifier_engine': '<textgrad.engin...
408,390,MMLU-CP,college_physics,Answer the following multiple choice question....,D,Here's how we can solve this problem:\n\n1. **...,The primary error lies in step 4 during the de...,<VERIFIED>The error lies in the squaring opera...,Here's how we can solve this problem:\n\n1. **...,,Here's how we can solve this problem:\n\n1. **...,A,True,None,13404.460693,6,5197,274,{'setup': {'verifier_engine': '<textgrad.engin...
409,411,MMLU-CP,college_physics,Answer the following multiple choice question....,B,The energy levels of a hydrogen-like atom are ...,The reduced mass correction is missing. The f...,<VERIFIED>The calculation uses the reduced mas...,The energy levels of a hydrogen-like atom are ...,,The energy levels of a hydrogen-like atom are ...,B,True,None,26820.566650,8,4766,638,{'setup': {'verifier_engine': '<textgrad.engin...
410,371,MMLU-CP,college_physics,Answer the following multiple choice question....,C,Here's how to figure out which harmonics survi...,The analysis of which harmonics survive is inc...,<VERIFIED>75 Hz (3rd harmonic of the open pipe...,Here's how to determine which harmonics surviv...,,Here's how to determine which harmonics surviv...,C,True,None,29135.683105,12,10866,888,{'setup': {'verifier_engine': '<textgrad.engin...
